##Library

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 896 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 11.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=ba322f7861fbd7dfa56bb8dc3731051b302c048bf91ef4f375d1f4c504036d3f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 31.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-tudloej0
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-tudloej0
     |████████████████████████████████| 132 kB 9.7 MB/s 
     |████████████████████████████████| 4.5 MB 60.5 MB/s 
     |████████████████████████████████| 4.4 MB 47.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.7 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 8.9 MB 46.8 MB/s 
     |████████████████████████████████| 127 kB 16.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=3eb371490c31114f906a2d2809f7ab9e19eb75c5e263d7d9803f4b32a324f62d
  Stored in directory: 

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
import numpy as np
# 데이터를 다루는 가장 기본이 되는 라이브러리
import pandas as pd
# 데이터를 엑셀의 형태로 보여주는 라이브러리
import keras
# 생성자를 임포트하기 위한 라이브러리
from tensorflow.keras import models, layers, optimizers, callbacks
# 케라스는 인공신경망을 쉽게 사용할 수 있게 개발된 라이브러리
import seaborn as sns
# 시각화 관련 라이브러리
import matplotlib.pyplot as plt
# 그래프 관련 라이브러리
from sklearn.metrics import confusion_matrix
# 분류 문제에 사용되는 컨퓨전 행렬 함수
from sklearn import preprocessing
# scikit-learn의 전처리 라이브러리
from tensorflow.keras.utils import to_categorical
# 범주형 변수를 one-hot encoding 해주는 함수

In [6]:

from collections import Counter
# 단어의 빈도를 계산하기 위한 라이브러리
import transformers
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문장을 인덱스로 변환하기 위한 라이브러리

import os
# 시스템 제어를 돕는 라이브러리
os.environ['CUDA_VISIBLE_DEVICES']='0'
# GPU가 여럿 존재하는 환경에서 0번 GPU만을 사용

In [7]:
# 데이터 로드를 위해 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Data Load & Preprocessing##

In [14]:
# # csv 데이터 불러오기
# preprocessed_train = pd.read_csv('/content/drive/MyDrive/산업인공지능/박시우/preprocessed_train.CSV', sep=',', engine = 'python', encoding = 'cp949', index_col=0)
# preprocessed_test = pd.read_csv('/content/drive/MyDrive/산업인공지능/박시우/preprocessed_test.CSV', sep=',', engine = 'python', encoding = 'cp949', index_col=0)

In [20]:
# # csv 데이터 파일 형식 tsv로 변환
# preprocessed_train.to_csv('/content/drive/MyDrive/산업인공지능/박시우/preprocessed_train.tsv',sep='\t',encoding='cp949', index=False)
# preprocessed_test.to_csv('/content/drive/MyDrive/산업인공지능/박시우/preprocessed_test.tsv',sep='\t',encoding='cp949', index=False)

In [21]:
# tsv 데이터 로드
train_data =nlp.data.TSVDataset('/content/drive/MyDrive/산업인공지능/박시우/preprocessed_train.tsv', field_indices = [0,1],encoding='cp949', num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/산업인공지능/박시우/preprocessed_test.tsv', field_indices = [0,1],encoding='cp949', num_discard_samples=1)

In [22]:
# 데이터 량 -> LSTM & CNN model과 동일한 데이터 량 확인
row_train, column = np.shape(train_data)
row_test, column = np.shape(dataset_test)
print('train data: {}, test data: {}, # of classes: {}'.format(row_train, row_test, 3))

train data: 12158, test data: 3737, # of classes: 3


##BERT model

In [23]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [24]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [26]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [27]:
train_data[0]

['일      안      하는      시간      쉬      싶어서      그런           게      아닐까', '0']

In [28]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3, ##클래스 수##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
device = torch.device("cuda") #gpu 사용
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [31]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss_ = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation loss {} validation acc {}".format(e+1,loss_.data.cpu().numpy() ,test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/190 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.191814661026001 train acc 0.265625
epoch 1 train acc 0.6069476867572156


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 validation loss 0.6911178827285767 validation acc 0.755614406779661


  0%|          | 0/190 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2101213037967682 train acc 0.890625
epoch 5 train acc 0.9308255517826824


  0%|          | 0/59 [00:00<?, ?it/s]

epoch 5 validation loss 0.6799623966217041 validation acc 0.7553495762711864
